# Genentech Cervical Cancer - Feature Selection Drugs

https://www.kaggle.com/c/cervical-cancer-screening/

In [2]:
# imports
import sys # for stderr
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# settings 
%logstop
%logstart  -o 'cc_feature_selection.log' rotate
plt.style.use('ggplot')
# constants
# plt.rcParams['figure.figsize'] = (10.0, 10.0)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_columns', 50)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : cc_feature_selection.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : False
State          : active


In [4]:
# versions 
import sys
print(pd.datetime.now())
print('Python: '+sys.version)
print('numpy: '+np.__version__)
print('pandas: '+pd.__version__)
print('sklearn: '+skl.__version__)

2016-01-29 16:05:42.335769
Python: 2.7.11 |Anaconda 2.4.0 (x86_64)| (default, Dec  6 2015, 18:57:58) 
[GCC 4.2.1 (Apple Inc. build 5577)]
numpy: 1.10.2
pandas: 0.17.1
sklearn: 0.17


In [4]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://paulperry:@localhost:5432/ccancer')

In [5]:
from pyace import ace

## Load 

In [6]:
fdir = './features/'

In [7]:
train_file = './input/patients_train.csv.gz'
train = pd.read_csv(train_file)
train.drop('patient_gender', axis=1, inplace=True)
train.set_index('patient_id', inplace=True)
train[:3]

,patient_age_group,patient_state,ethinicity,household_income,education_level,is_screener
patient_id,,,,,,
336201912,51-53,SD,ALL OTHER,UNKNOWN,UNKNOWN,1
373337412,21-23,LA,ALL OTHER,UNKNOWN,UNKNOWN,1
368845412,12-14,OH,ALL OTHER,UNKNOWN,UNKNOWN,0


## Run

In [8]:
import datetime
start = datetime.datetime.now()
print(start)

2016-01-29 07:33:32.975777


In [9]:
tab = pd.read_csv(fdir+'train_obgyn_drug.csv')
#tab.set_index('patient_id', inplace=True)
tab.shape

(8861163, 5)

In [10]:
tab.drop_duplicates(['patient_id','drug_id'], inplace=True)
tab.shape

(3166274, 5)

In [11]:
drugs = tab.drug_id.unique()
len(drugs)

24296

In [24]:
cat_cols = ['patient_age_group','patient_state','ethinicity','household_income','education_level']

In [14]:
print(tab_pivot.shape)
tab_pivot[:10]

(529335, 1000)


drug_id,3191,4588,4635,10201,10213,15798,15855,19619,19662,19664,...,401471,401472,401473,402050,402151,402301,402829,403450,403747,403907
patient_id,,,,,,,,,,,,,,,,,,,,,
84548626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84549151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84549169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84550136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84550482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84550563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84550735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
tab_dummy = (tab_pivot[:] > 0).astype(int)

# tab_dummy[tab_dummy.iloc[:,0] > 0]

sums = tab_dummy.sum()
sums.sort_values(ascending=False, inplace=True)
tab_sub = tab_pivot.loc[:,sums[sums > 20].index] 

In [79]:
step = 1000
for i in range(0,len(drugs),step):
    print(i, i+step, ':'), 
    subset = tab[tab.drug_id.isin(drugs[i:i+step])]
    tab_pivot = subset.pivot('patient_id','drug_id','drug_id')
    tab_dummy = (tab_pivot[:] > 0).astype(int)
    sums = tab_dummy.sum()
    sums.sort_values(ascending=False, inplace=True)
    tab_sub = tab_pivot.loc[:,sums[sums > 100].index]
    if tab_sub.shape[1] < 1:
        continue
    print(tab_sub.shape)
    dfall = pd.merge(train, tab_sub, left_index=True, right_index=True, how='left')
    ranks = ace(dfall, 'is_screener', cat_cols=[])
    df_ranks = pd.DataFrame(ranks, index=dfall.columns, columns=['ace','mean'])
    df_ranks = df_ranks.sort_values(by='ace', ascending=False)
    top_ranks = df_ranks[df_ranks.ace > 0]
    if top_ranks.shape[0] > 5:
        break
    gc.collect()

0
(529335, 807) 1000
(214411, 591) 2000
(220643, 492) 3000
(142642, 423) 4000
(92028, 307) 5000
(49442, 164) 6000
(33629, 84) 7000
(25010, 47) 8000
(18508, 16) 9000
(13998, 6) 10000
(10614, 2) 11000
('nothing at: ', 11000)
12000
('nothing at: ', 12000)
13000
('nothing at: ', 13000)
14000
('nothing at: ', 14000)
15000
('nothing at: ', 15000)
16000
('nothing at: ', 16000)
17000
('nothing at: ', 17000)
18000
('nothing at: ', 18000)
19000
('nothing at: ', 19000)
20000
('nothing at: ', 20000)
21000
('nothing at: ', 21000)
22000
('nothing at: ', 22000)
23000
('nothing at: ', 23000)
24000
('nothing at: ', 24000)


In [78]:
tab_sub.shape

(298, 0)

In [ ]:
nnn = 1

In [ ]:
break

In [ ]:
top_ranks.to_csv('drug_ranks_'+str(nnn)+'.csv')

In [ ]:
end   = datetime.datetime.now()
print('run time: '+str(end-start)+' at: '+str(end))

In [ ]:
break

In [ ]:
top_ranks = pd.read_csv('diagnosis_ranks_15.csv')
top_ranks.set_index('Unnamed: 0', inplace=True)
top_ranks[:10]

In [ ]:
qlist = list(top_ranks[top_ranks.ace > 0.003891].index)
for c in cat_cols: 
    qlist.remove(c)
qlist_str = "('"+qlist[0]+"'"
for c in qlist[1:]: 
    qlist_str=qlist_str+",'"+c+"'"
qlist_str=qlist_str+')'
qlist_str

In [ ]:
q = 'select * from diagnosis_code where diagnosis_code in '+qlist_str

diag_codes = pd.read_sql_query(q, engine)
diag_codes

In [ ]:
diag_codes.to_csv('diagnosis_top_codes.csv', mode='a', header=False)

In [ ]:
qlist

In [ ]:
tab[qlist].to_csv('diagnosis_top_'+str(nnn)+'.csv')

## physician codes

In [5]:
physician = ['GO','GYN','OBG','OBS','OCC','ON','OTO','PCP','REN', 'FPP', 'PLN','FPG', 'FM', 'FP']
phy_codes = pd.read_csv('features/physician_specialty_codes.csv')

In [7]:
phy_codes[phy_codes.specialty_code.isin(physician)]

,specialty_code,specialty_description
8,REN,REPRODUCTIVE ENDOCRINOLOGY
23,GO,GYNECOLOGICAL ONCOLOGY
30,FPP,FAMILY PRACTICE/PSYCHIATRY
32,OTO,OTOLARYNGOLOGY
57,PLN,FAMILY PLANNING NURSE PRACTITIONER
67,GYN,GYNECOLOGY
84,ON,ONCOLOGY MEDICAL
98,PCP,"PATHOLOGY, CYTOPATHOLOGY"
103,OBG,OBSTETRICS AND GYNECOLOGY
109,OCC,"OBSTETRICS/GYNECOLOGY, CRITICAL CARE"


In [ ]:
FP = FP , FM, PLN
OBG = OBG, GYN, REN, OBS
NOT = OTO, PCP, GO, FPP, OCC, FPG, ON 

In [8]:
proc_years = pd.read_csv('./features/procedure/procedure_years.csv')
proc_years.shape

(2859610, 6)

In [9]:
proc_years[:10]

,patient_id,len,year_min,year_max,year_delta,year_delta_2014
0,84548607,131,2008,2014,6,6
1,84548626,47,2008,2014,6,6
2,84548666,29,2008,2014,6,6
3,84548780,84,2008,2014,6,6
4,84548805,124,2008,2014,6,6
5,84548821,45,2009,2014,5,5
6,84548841,47,2008,2014,6,6
7,84548915,311,2008,2014,6,6
8,84549017,106,2008,2014,6,6
9,84549024,65,2008,2014,6,6


In [10]:
proc_years.describe()

,patient_id,len,year_min,year_max,year_delta,year_delta_2014
count,2.859610e+06,2859610.000000,2859610.000000,2859610.000000,2859610.000000,2859610.000000
mean,2.065658e+08,106.099882,2008.319357,2013.881815,5.562458,5.680643
std,1.205161e+08,141.596791,0.622024,0.343196,0.729965,0.622024
min,8.454861e+07,1.000000,2008.000000,2008.000000,0.000000,0.000000
25%,1.172777e+08,42.000000,2008.000000,2014.000000,5.000000,6.000000
50%,1.712824e+08,72.000000,2008.000000,2014.000000,6.000000,6.000000
75%,2.449437e+08,124.000000,2008.000000,2014.000000,6.000000,6.000000
max,8.876579e+08,7826.000000,2014.000000,2014.000000,6.000000,6.000000


In [15]:
proc_years.year_min.max()

2014

In [16]:
# avg_visits per year.